# core

> Defines structure of ProcessingStrategies and ProcessingObjects:

In [ ]:
#| default_exp core

The following two classes, `ProcessingObject` and `ProcessingStrategy`, provide the blueprint for all processing strategies and objects that are used throughout the *findmycells* package. As you can see in the corresponding processing step modules (i.e. "preprocess", "segment", or "quantify"), these abstract base classes provide the basic structure of the more specific objects and strategies in each of these modules (i.e. `QuantificationObject` and `QuantificationStrategy` within the "quantify" module inherit from `ProcessingObject` and `ProcessingStrategy`, respectively). This makes these two classes of course highly relevant for any developer, but not so much for users, even if they want to use the API instead of the GUI.

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from abc import ABC, abstractmethod
from findmycells.database import Database
from typing import List, Dict

In [ ]:
#| export

class ProcessingObject(ABC):
    
    """
    Abstract base class (inherits from ABC) that defines the general structure of `ProcessingObjects` in findmycells.
    A `ProcessingObject` combines all information needed for the corresponding processing step, 
    i.e. what files are supposed to be processed & how. It also interfaces to the database of the
    project, such that it can automatically update the database with the latest progress.
    """
    
    def __init__(self, database: Database, file_ids: List, strategies: List) -> None:
        """
        ----------
        Parameters:
        
            database (findmycells.database.Database): The database of the findmycells project.
            
            file_ids (List[str]): A list with the file_ids of all files that need to be processed.
            
            strategies (List[findmycells.core.ProcessingStrategy]): A list of all ProcessingStrategies that shall be run on the files defined in file_ids.
        """
        
        self.database = database
        self.file_ids = file_ids
        self.strategies = strategies # strategies is a list of ProcessingStrategies (can of course also be just a single strategy)
        # additional attributes can be added in the respective subclasses

    @property
    @abstractmethod
    def processing_type(self) -> str:
        """
        Abstract method that requires its subclasses to define the `processing_type`
        as a property of the class. Thus, this will be specified in each individual 
        processing module (e.g. the "preprocess" or "quantify" modules). It will be used
        in the database to keep track of the processing progress of the project.
        ----------
        Returns:
        
            processing_type (str): processing type identifier (e.g. "proprocessing" or "segmentation", unique for each processing step module)
        """
        pass


    @abstractmethod
    def add_processing_specific_infos_to_updates(self, updates: Dict) -> Dict:
        """
        Abstract method that that requires its subclasses to define what updates need to be
        passed to the database, in addition to those that are already covered by the corresponding
        ProcessingStrategies or the "self.update_database()" method. If there are no more 
        information to add, simply return the input 'updates' dictionary without any alterations.
        ----------
        Parameters:
        
            updates (dict): A dictionary with updates that need to be passed to the database.
        ----------
        Returns:
        
            updates (dict): A dictionary with all updates that need to be passed to the database. 
        
        """
        return updates
    
    
    def run_all_strategies(self) -> None:
        """
        Runs all ProcessingStrategies that were passed upon initialization (i.e. self.strategies).
        For this, the corresponding ProcessingStrategy objects will be initialized and their ".run()"
        method will be called, while passing "self" as "processing_object". Finally, it updates the
        database and deletes the ProcessingStrategy object to clear it from memory.
        """
        for strategy in self.strategies:
            processing_strategy = strategy()
            self = processing_strategy.run(processing_object = self)
            self = processing_strategy.update_database(processing_object = self)
            del processing_strategy


    def update_database(self) -> None:
        """
        For each microscopy file that had to be processed (self.file_ids), the database
        will be updated with the respective processing progress information. Interfaces
        back to the abstract method "self.add_processing_specific_infos_to_updates()" that
        enables the corresponding subclasses to add more specific details before triggering
        the update method of the database.
        """
        for file_id in self.file_ids:
            updates = dict()
            updates[f'{self.processing_type}_completed'] = True
            updates = self.add_processing_specific_infos_to_updates(updates = updates)
            self.database.update_file_infos(file_id = file_id, updates = updates)

Subclasses that inherit from `ProcessingObject` need to implement the following two abstract methods:

In [ ]:
show_doc(ProcessingObject.processing_type)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L29){target="_blank" style="float:right; font-size:smaller"}

### ProcessingObject.processing_type

>      ProcessingObject.processing_type ()

Abstract method that requires its subclasses to define the `processing_type`
as a property of the class. Thus, this will be specified in each individual 
processing module (e.g. the "preprocess" or "quantify" modules). It will be used
in the database to keep track of the processing progress of the project.
----------
Returns:

processing_type (str): processing type identifier (e.g. "proprocessing" or "segmentation", unique for each processing step module)

In [ ]:
show_doc(ProcessingObject.add_processing_specific_infos_to_updates)

/home/ds/anaconda3/envs/findmycells/lib/python3.9/site-packages/fastcore/docscrape.py:225: UserWarning: potentially wrong underline length... 
updates (dict): A dictionary with updates that need to be passed to the database. 
---------- in 
Abstract method that that requires its subclasses to define what updates need to be
passed to the database, in addition to those that are already covered by the corresponding...
  else: warn(msg)


---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L35){target="_blank" style="float:right; font-size:smaller"}

### ProcessingObject.add_processing_specific_infos_to_updates

>      ProcessingObject.add_processing_specific_infos_to_updates (updates:Dict)

Abstract method that that requires its subclasses to define what updates need to be
passed to the database, in addition to those that are already covered by the corresponding
ProcessingStrategies or the "self.update_database()" method. If there are no more 
information to add, simply return the input 'updates' dictionary without any alterations.
----------
Parameters:

    updates (dict): A dictionary with updates that need to be passed to the database.
----------
Returns:

    updates (dict): A dictionary with all updates that need to be passed to the database.

In addition, `ProcessingObject` defines two core functions that will be called on all its subclasses, which are:

In [ ]:
show_doc(ProcessingObject.run_all_strategies)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L42){target="_blank" style="float:right; font-size:smaller"}

### ProcessingObject.run_all_strategies

>      ProcessingObject.run_all_strategies ()

Runs all ProcessingStrategies that were passed upon initialization (i.e. self.strategies).
For this, the corresponding ProcessingStrategy objects will be initialized and their ".run()"
method will be called, while passing "self" as "processing_object". Finally, it updates the
database and deletes the ProcessingStrategy object to clear it from memory.

In [ ]:
show_doc(ProcessingObject.update_database)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L50){target="_blank" style="float:right; font-size:smaller"}

### ProcessingObject.update_database

>      ProcessingObject.update_database ()

For each microscopy file that had to be processed (self.file_ids), the database
will be updated with the respective processing progress information. Interfaces
back to the abstract method "self.add_processing_specific_infos_to_updates()" that
enables the corresponding subclasses to add more specific details before triggering
the update method of the database.

In [ ]:
#| export

class ProcessingStrategy(ABC):
    
    """
    Abstract base class that defines the general structure of `ProcessingStrategies` in findmycells.
    A `ProcessingStrategy` combines all functions that are required for one particular processing step, 
    e.g. `ConvertTo8Bit` is a `ProcessingStrategy` in the "preprocess" module and converts the corresponding
    images into 8-bit.
    """

    @property
    @abstractmethod
    def processing_type(self):
        # has to be any of these: 'preprocessing', 'segmentation', 'quantification', 'inspection'
        pass


    @abstractmethod
    def run(self, processing_object: ProcessingObject) -> ProcessingObject:
        # process the processing_object
        return processing_object

    
    @abstractmethod
    def add_strategy_specific_infos_to_updates(self, updates: Dict) -> Dict:
        # add all ProcessingStrategy specifc information to the update dictionary
        # or simply return updates right away if there are no information to add
        return updates
    def update_database(self, processing_object: ProcessingObject) -> ProcessingObject:
        for file_id in processing_object.file_ids:
            updates = dict()
            step_index = self.determine_correct_step_index(database = processing_object.database, file_id = file_id)
            updates[f'{self.processing_type}_step_{str(step_index).zfill(2)}'] = self.strategy_name
            updates = self.add_strategy_specific_infos_to_updates(updates = updates)
            processing_object.database.update_file_infos(file_id = file_id, updates = updates)
        return processing_object


    @property
    def strategy_name(self):
        return self.__class__.__name__ 
            
    
    def determine_correct_step_index(self, database: Database, file_id: str) -> int:
        file_infos = database.get_file_infos(identifier = file_id)
        previous_step_indices_of_same_processing_type = []
        for key, value in file_infos.items():
            if f'{self.processing_type}_step_' in key:
                if value != None: # to ensure that this file_id was actually already processed
                    step_index = int(key[key.rfind('_') + 1 :])
                    previous_step_indices_of_same_processing_type.append(step_index)
        if len(previous_step_indices_of_same_processing_type) > 0:
            correct_step_index = max(previous_step_indices_of_same_processing_type) + 1
        else:
            correct_step_index = 0
        return correct_step_index

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()